In [1]:
import pandas as pd

In [2]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [23]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [191]:
stocks.head(15)

,raison_sociale,rubric_id,quantite,unite
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,GAEC PORC D'ALGRES,3660,860.0,NaN
2,GAEC PORC D'ALGRES,2102,860.0,NaN
3,GAEC FERME H ET L,2101,105.0,NaN
4,GAEC FERME H ET L,2101,150.0,NaN
5,GAEC FERME H ET L,2101,210.0,u
6,BINOUDIERE (GAEC DE LA),2102,498.0,Animaux-Eq
7,BINOUDIERE (GAEC DE LA),3660,498.0,NaN
8,BINOUDIERE (GAEC DE LA),2101,60.0,u
9,BINOUDIERE (GAEC DE LA),2101,80.0,u


In [198]:
# réponse question 3
df = farms.merge(stocks, 'inner', left_on= 'name', right_on= 'raison_sociale')
df = df[['name', 'departement', 'type' ,'rubric_id', 'quantite', 'unite']]
df = df.convert_dtypes()
df.dropna()
df.groupby('name')

dfD = df[df.departement == 44].reset_index()
print(dfD.head(15))

# fonction pour avoir la dernière rubrique d'une ferme et avoir son nombre d'animaux déclaré le plus récent
def getNbAnimalsOfFarm(rub_id : int, name : str, dfD) -> int:
    dfD = dfD[(dfD.name == name) & (dfD.rubric_id == rub_id)] 
    if len(dfD.rubric_id) == 0:
        return dfD.at[0, 'quantite']
    else:
        return dfD['quantite'].iloc[-1]
    

# function pour avoir l'id de la rubrique selon le type de la ferme
def getIdRub(type: str) -> str:
    if type == 'BOVINS':
        return '2101'
    elif type == 'PORCS':
        return '2102'
    else :
        return '2111'
    


def getBiggerBreedingDep(dep : int) -> str:
    nbBovins :int = 0
    nbPorcs :int = 0
    nbVolailles :int = 0
    nbTmp : int 
    dfDep = df[df.departement == dep].reset_index()
    name : str = ''

    print(dfDep.head(15))

    for i in dfDep.index:
        if i == 0:
            name = dfDep.at[i, 'name']
            nbTmp = getNbAnimalsOfFarm(getIdRub(dfDep.at[i, 'type']), name, dfDep)
        elif name != dfDep.at[i, 'name']:
            name = dfDep.at[i, 'name']
            nbTmp = getNbAnimalsOfFarm(getIdRub(dfDep.at[i, 'type']), name, dfDep)

        if dfDep.at[i, 'type'] == 'BOVINS':
            nbBovins += nbTmp
        elif dfDep.at[i, 'type'] == 'PORCS':
            nbPorcs += nbTmp
        elif dfDep.at[i, 'unite'] == 'Animaux-Eq':
            nbVolailles += nbTmp * 0.01 
            print('volaille' + nbVolailles)
        else:
            nbVolailles += nbTmp      
        nbTmp = 0

    print(nbPorcs)
    print(nbBovins)
    print(nbVolailles)

    if max(nbBovins, nbPorcs, nbVolailles) == nbBovins:
        return 'Bovin'
    elif max(nbBovins, nbPorcs, nbVolailles) == nbPorcs:
        return 'Porc'
    else:
        return 'Volaille'

print(getBiggerBreedingDep(74))
    
reponse3 = pd.DataFrame({'departement' : [], 'reponse' : []})

reponse3.head()

    index                   name  departement       type rubric_id  quantite  \
0     165  GAEC DES QUATRE VENTS           44     BOVINS      2101     250.0   
1     237     GAEC DES FLEURIAIS           44     BOVINS      2101     120.0   
2     238     GAEC DES FLEURIAIS           44     BOVINS      2101     136.0   
3     239     GAEC DES FLEURIAIS           44     BOVINS      3660   57000.0   
4     240     GAEC DES FLEURIAIS           44     BOVINS      2101     120.0   
5     241       GAEC DES VALLEES           44      PORCS      3660    1095.0   
6     242       GAEC DES VALLEES           44      PORCS      2102    1119.0   
7     394         SOULARD Ernest           44  VOLAILLES      2111   37600.0   
8     395         SOULARD Ernest           44  VOLAILLES      2111   57600.0   
9     496          DE LA HAUTURE           44      PORCS      3660    1796.0   
10    497          DE LA HAUTURE           44      PORCS      2102    2937.0   
11   1363         EARL BELVEDERE        

,departement,reponse
